In [1]:
# Regression using ANN in PyTorch

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
# Load the dataset
data = pd.read_csv("C:\\Users\\ngajula\\AppData\\Local\\Programs\\Python\\Python312\\Scripts\\ADL_CV\\Day7\\50_Startups.csv")
data.dropna(inplace=True)

In [6]:
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [8]:
data.dtypes

R&D Spend          float64
Administration     float64
Marketing Spend    float64
State               object
Profit             float64
dtype: object

In [9]:
cols = list(data.select_dtypes(np.object_).columns)
data[cols] = data[cols].astype('string')
data.dtypes

R&D Spend                 float64
Administration            float64
Marketing Spend           float64
State              string[python]
Profit                    float64
dtype: object

In [10]:
data['State'].unique()

<StringArray>
['New York', 'California', 'Florida']
Length: 3, dtype: string

In [11]:
State_mapping = {'New York': 1, 'California': 2, 'Florida':3}
data['State'] = data['State'].map(State_mapping)

In [13]:
data['State'].unique()

array([1, 2, 3], dtype=int64)

In [14]:
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,1,192261.83
1,162597.70,151377.59,443898.53,2,191792.06
2,153441.51,101145.55,407934.54,3,191050.39
3,144372.41,118671.85,383199.62,1,182901.99
4,142107.34,91391.77,366168.42,3,166187.94


In [20]:
# Features and label
X = data[['R&D Spend','Administration','Marketing Spend','State']].values  # input feature
y = data['Profit'].values  # target variable

In [21]:
# Feature Scaling
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [22]:
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1))  # Reshape y for scaling

In [23]:
# Convert to tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [24]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# Send data to device
X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

In [29]:
# Define a simple ANN model for regression
class RegressionANN(nn.Module):
    def __init__(self):
        super(RegressionANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x):
        return self.model(x)

In [30]:
# Instantiate model, loss function and optimizer
model = RegressionANN().to(device)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [31]:
# Training loop
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Calculate training R2 Score
    with torch.no_grad():
        train_preds = scaler_y.inverse_transform(outputs.cpu().numpy())
        y_train_actual = scaler_y.inverse_transform(y_train.cpu().numpy())
        train_r2 = r2_score(y_train_actual, train_preds)

    # Evaluation on validation set
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)

        val_preds = scaler_y.inverse_transform(val_outputs.cpu().numpy())
        y_val_actual = scaler_y.inverse_transform(y_test.cpu().numpy())
        val_r2 = r2_score(y_val_actual, val_preds)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}")


Epoch [10/100], Train Loss: 0.9917, Val Loss: 0.5301, Train R2: 0.0828, Val R2: -0.0423
Epoch [20/100], Train Loss: 0.6532, Val Loss: 0.5309, Train R2: 0.3959, Val R2: -0.0438
Epoch [30/100], Train Loss: 0.2947, Val Loss: 0.3091, Train R2: 0.7274, Val R2: 0.3923
Epoch [40/100], Train Loss: 0.1445, Val Loss: 0.2210, Train R2: 0.8664, Val R2: 0.5655
Epoch [50/100], Train Loss: 0.1158, Val Loss: 0.2711, Train R2: 0.8929, Val R2: 0.4671
Epoch [60/100], Train Loss: 0.0886, Val Loss: 0.1675, Train R2: 0.9180, Val R2: 0.6706
Epoch [70/100], Train Loss: 0.0800, Val Loss: 0.1148, Train R2: 0.9260, Val R2: 0.7742
Epoch [80/100], Train Loss: 0.0694, Val Loss: 0.1057, Train R2: 0.9358, Val R2: 0.7922
Epoch [90/100], Train Loss: 0.0605, Val Loss: 0.0880, Train R2: 0.9440, Val R2: 0.8271
Epoch [100/100], Train Loss: 0.0512, Val Loss: 0.0664, Train R2: 0.9527, Val R2: 0.8694


In [32]:
# Final evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predictions = predictions.cpu().numpy()
    y_test_np = y_test.cpu().numpy()

    # Inverse scaling
    predictions = scaler_y.inverse_transform(predictions)
    y_actual = scaler_y.inverse_transform(y_test_np)

    mse = mean_squared_error(y_actual, predictions)
    r2 = r2_score(y_actual, predictions)

In [33]:
print("\nTest MSE:", mse)
print("R2 Score:", r2)


Test MSE: 105791040.0
R2 Score: 0.8693602282820636
